In [3]:
import scrapy

class bookSpider(scrapy.Spider):
    
    name = 'books'
    start_urls = ['']
    
    def parse(self, response):
        for book in response.css('article.product_pod'):
            name = book.xpath('./h3/a/@title').extract_first()
            price = book.css('p.price_color::text').extract_first()
            
            yield{
                'name':name,
                'price':price
            }
            
        next_url = reponse.css('ul.pager li.next a::attr(href)').extract_first()
        
        if next_url:
            next_url = reponse.urljoin(next_url)
            yield scrapy.Request(next_url, callback = self.parse)

In [4]:
from scrapy.selector import Selector

text = '''

    <html>
        <body>
            <h1>Hello World</h1>
            <h1>Hello Scrapy</h1>
            <b>Hello python></b>
            <ul>
                <li>C++</li>
                <li>Java</li>
                <li>Python</li>
            </ul>
        </body>
    </html>
    
'''

selector = Selector(text = text) #用Selector讀html
print(selector)


<Selector xpath=None data='<html>\n        <body>\n            <h1...'>


In [6]:
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

body ='''

<html>
        <body>
            <h1>Hello World</h1>
            <h1>Hello Scrapy</h1>
            <b>Hello python></b>
            <ul>
                <li>C++</li>
                <li>Java</li>
                <li>Python</li>
            </ul>
        </body>
    </html>

'''


response = HtmlResponse(url='http://www.example.com', body = body, encoding = 'utf-8')
selector = Selector(response = response)
print(selector)



<Selector xpath=None data='<html>\n        <body>\n            <h1...'>


In [10]:
selector_list = selector.xpath('//h1')
print(selector_list)

print('='*100)

for sel in selector_list:
    print(sel.xpath('./text()'))

print('='*100)

print(selector_list.xpath('./text()'))

print('='*100)

print(selector.xpath('//ul').css('li').xpath('./text()'))

[<Selector xpath='//h1' data='<h1>Hello World</h1>'>, <Selector xpath='//h1' data='<h1>Hello Scrapy</h1>'>]
[<Selector xpath='./text()' data='Hello World'>]
[<Selector xpath='./text()' data='Hello Scrapy'>]
[<Selector xpath='./text()' data='Hello World'>, <Selector xpath='./text()' data='Hello Scrapy'>]
[<Selector xpath='./text()' data='C++'>, <Selector xpath='./text()' data='Java'>, <Selector xpath='./text()' data='Python'>]


In [49]:
s1 = selector.xpath('.//li')
print(s1)
print('-'*50)
print(s1[0].extract())

print('='*50)

s1 = selector.xpath('.//li/text()')
print(s1)
print('-'*50)
print(s1[0].extract())

print('='*50)
s1 = selector.xpath('.//li')
print(s1.extract())
print(s1.extract_first())

[<Selector xpath='.//li' data='<li>Python學習手冊 <b>價格：99.00元</b></li>'>, <Selector xpath='.//li' data='<li>Python核心程式設計 <b>價格：88.00元</b></li>'>, <Selector xpath='.//li' data='<li>Python基礎教學 <b>價格：80.00元</b></li>'>]
--------------------------------------------------
<li>Python學習手冊 <b>價格：99.00元</b></li>
[<Selector xpath='.//li/text()' data='Python學習手冊 '>, <Selector xpath='.//li/text()' data='Python核心程式設計 '>, <Selector xpath='.//li/text()' data='Python基礎教學 '>]
--------------------------------------------------
Python學習手冊 
['<li>Python學習手冊 <b>價格：99.00元</b></li>', '<li>Python核心程式設計 <b>價格：88.00元</b></li>', '<li>Python基礎教學 <b>價格：80.00元</b></li>']
<li>Python學習手冊 <b>價格：99.00元</b></li>


In [29]:
text = '''
    <ul>
        <li>Python學習手冊 <b>價格：99.00元</b></li>
        <li>Python核心程式設計 <b>價格：88.00元</b></li>
        <li>Python基礎教學 <b>價格：80.00元</b></li>
    </ul>
'''

selector = Selector(text = text)
price = selector.xpath('.//li/b/text()')
print(price)
print('='*50)
print(price.extract())
print('='*50)
print(price.re('\d+\.\d+'))  #re自帶extract效果
print('='*50)
print(price.re_first('\d+\.\d+'))

[<Selector xpath='.//li/b/text()' data='價格：99.00元'>, <Selector xpath='.//li/b/text()' data='價格：88.00元'>, <Selector xpath='.//li/b/text()' data='價格：80.00元'>]
['價格：99.00元', '價格：88.00元', '價格：80.00元']
['99.00', '88.00', '80.00']
99.00


In [35]:
#response 不需丟進selector

from scrapy.http import HtmlResponse

body = '''
    <html>
        <body>
            <h1>Hello World</h1>
            <h1>Hello Scrapy</h1>
            <b>Hello python></b>
            <ul>
                <li>C++</li>
                <li>Java</li>
                <li>Python</li>
            </ul>
        </body>
    </html>
'''

response = HtmlResponse(url='http://www.example.com', body = body, encoding='utf8')
print(response.selector)
print('-'*50)
print(response.xpath('.//h1/text()').extract())
print(response.xpath('.//li/text()').extract())


<Selector xpath=None data='<html>\n        <body>\n            <h1...'>
--------------------------------------------------
['Hello World', 'Hello Scrapy']
['C++', 'Java', 'Python']


In [153]:
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

body = '''
    <html>
        <head>
            <base href='http://example.com/' />
            <title>Example website</title>
        </head>
        <body>
            <div id='images'>
                <a href = 'image1.html'>Name: Image I<br/><img src='image1.jpg' /></a>
                <a href = 'image2.html'>Name: Image 2<br/><img src='image2.jpg' /></a>
                <a href = 'image3.html'>Name: Image 3<br/><img src='image3.jpg' /></a>
                <a href = 'image4.html'>Name: Image 4<br/><img src='image4.jpg' /></a>
                <a href = 'image5.html'>Name: Image 5<br/><img src='image5.jpg' /></a>
            </div>
            <img src='image1.jpg' />
        </body>
        
        <body>
            <div id='images'>
                <a href = 'image99.html'>Name: Image 99<br/><img src='image99.jpg' /></a>
                <a href = 'image100.html'>Name: Image 100<br/><img src='image100.jpg' /></a>
            </div>
        </body>
    </html>
'''

response = HtmlResponse(url = 'http://www.example.com',body = body, encoding='utf8')
print(response.xpath('/html'))
print(response.xpath('/html/head'))


[<Selector xpath='/html' data='<html>\n        <head>\n            <ba...'>]
[<Selector xpath='/html/head' data='<head>\n            <base href="http:/...'>]


In [150]:
# # 選取 div之下的所有a (指定特定路徑中的 a)
# print(response.xpath('//a').extract())
# print(response.xpath('./body/div/a/text()').extract())

for item in response.xpath('//a').extract():
    print(item)

<a href="image1.html">Name: Image I<br><img src="image1.jpg"></a>
<a href="image2.html">Name: Image 2<br><img src="image2.jpg"></a>
<a href="image3.html">Name: Image 3<br><img src="image3.jpg"></a>
<a href="image4.html">Name: Image 4<br><img src="image4.jpg"></a>
<a href="image5.html">Name: Image 5<br><img src="image5.jpg"></a>
<a href="image1.html">Name: Image I<br><img src="image1.jpg"></a>
<a href="image2.html">Name: Image 2<br><img src="image2.jpg"></a>


In [134]:
for item in response.xpath('./body//img'):
    print(item)

<Selector xpath='./body//img' data='<img src="image1.jpg">'>
<Selector xpath='./body//img' data='<img src="image2.jpg">'>
<Selector xpath='./body//img' data='<img src="image3.jpg">'>
<Selector xpath='./body//img' data='<img src="image4.jpg">'>
<Selector xpath='./body//img' data='<img src="image5.jpg">'>
<Selector xpath='./body//img' data='<img src="image1.jpg">'>
<Selector xpath='./body//img' data='<img src="image1.jpg">'>
<Selector xpath='./body//img' data='<img src="image2.jpg">'>


In [113]:
# # 選取html下的所有元素子節點 (daughter tags)
# for item in response.xpath('/html/*'):
#     print(item.extract)

# print('-'*50)

# # 選取html下的所有元素子與孫節點 (daughter tags)
# for item in response.xpath('/html/body/div//*'):
#     print(item.extract)

In [146]:
# 選取所有div的孫節點img
for item in response.xpath('//div/*/img'):
    print(item)


# 選取所有img的scr屬性
for item in response.xpath('//a/@href'):
    print(item.extract())

<Selector xpath='//div/*/img' data='<img src="image1.jpg">'>
<Selector xpath='//div/*/img' data='<img src="image2.jpg">'>
<Selector xpath='//div/*/img' data='<img src="image3.jpg">'>
<Selector xpath='//div/*/img' data='<img src="image4.jpg">'>
<Selector xpath='//div/*/img' data='<img src="image5.jpg">'>
<Selector xpath='//div/*/img' data='<img src="image1.jpg">'>
<Selector xpath='//div/*/img' data='<img src="image2.jpg">'>
image1.html
image2.html
image3.html
image4.html
image5.html
image1.html
image2.html


In [127]:
for item in response.xpath('//@href'):
    print(item)

<Selector xpath='//@href' data='http://example.com/'>
<Selector xpath='//@href' data='image1.html'>
<Selector xpath='//@href' data='image2.html'>
<Selector xpath='//@href' data='image3.html'>
<Selector xpath='//@href' data='image4.html'>
<Selector xpath='//@href' data='image5.html'>
<Selector xpath='//@href' data='image1.html'>
<Selector xpath='//@href' data='image2.html'>


In [163]:
#將index為1的都抓出來
for item in response.xpath('//a[1]'):
    print(item)

<Selector xpath='//a[1]' data='<a href="image1.html">Name: Image I<b...'>
<Selector xpath='//a[1]' data='<a href="image99.html">Name: Image 99...'>


In [162]:
item = response.xpath('//a')[0]
print(item)

<Selector xpath='//a' data='<a href="image1.html">Name: Image I<b...'>


In [170]:
# 選取所有a中的第3個
print(response.xpath('//a[3]'))

# 最後一個
print(response.xpath('//a[last()]').extract())

# 使用position，選前三個
print(response.xpath('//a[position()<=3]'))

#選取所有含有id屬性的div
print(response.xpath('//id[@id]'))

# 選取所有含id屬性且值為images的div
print(response.xpath('//div[@id="images"]'))

[<Selector xpath='//a[3]' data='<a href="image3.html">Name: Image 3<b...'>]
['<a href="image5.html">Name: Image 5<br><img src="image5.jpg"></a>', '<a href="image100.html">Name: Image 100<br><img src="image100.jpg"></a>']
[<Selector xpath='//a[position()<=3]' data='<a href="image1.html">Name: Image I<b...'>, <Selector xpath='//a[position()<=3]' data='<a href="image2.html">Name: Image 2<b...'>, <Selector xpath='//a[position()<=3]' data='<a href="image3.html">Name: Image 3<b...'>, <Selector xpath='//a[position()<=3]' data='<a href="image99.html">Name: Image 99...'>, <Selector xpath='//a[position()<=3]' data='<a href="image100.html">Name: Image 1...'>]
[]
[<Selector xpath='//div[@id="images"]' data='<div id="images">\n                <a ...'>, <Selector xpath='//div[@id="images"]' data='<div id="images">\n                <a ...'>]


In [172]:
# .xpath()中的 string()

from scrapy.selector import Selector
text = '<a href="#">Click here go to to the <strong>Next Page</strong></a>'
sel = Selector(text=text)
print(sel.xpath('string(/html/body/a/strong)').extract())
print(sel.xpath('/html/body/a/strong/text()').extract())

['Next Page']
['Next Page']


In [177]:
# 取得整個字串 Click here to go to the Next Page
# 因為上面的字串分屬不同的標籤(元素)之下

print(sel.xpath('//a//text()').extract())
print(sel.xpath('string(//a)').extract())

['Click here go to to the ', 'Next Page']
['Click here go to to the Next Page']


In [181]:
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

body = '''
    <html>
        <head>
            <base href='http://example.com/' />
            <title>Example website</title>
        <body>
            <div id="images-1" style='width: 1230px;'>
                <a href='image1.html'>Name: Image1 <br/><img src='imge1.jpg' /></a>
                <a href='image1.html'>Name: Image2 <br/><img src='imge2.jpg' /></a>
                <a href='image1.html'>Name: Image3 <br/><img src='imge3.jpg' /></a>
            </div>
            <div id='images-2' class='small'>
                <a href='image4.html'>Name: Image 4 <br/><imag src='image4.jpg' /></a>
                <a href='image5.html'>Name: Image 5 <br/><imag src='image5.jpg' /></a>
            </div>
        </body>
    </html>
'''

response = HtmlResponse(url='http://www.example.com',body=body,encoding='utf8')


In [203]:
# 選取所有的 img
print(response.css('img').extract())
print('='*100)
# 選取所有base和title
print(response.css('base, title').extract())
print('='*100)
# 選取div後代中的img (descendants)
print(response.css('div img'))
print('='*100)
# 選取body子元素中的div (daughters)
print(response.css('body>div'))
print('='*100)
# 選取包含style屬性的元素 attrs不用“”
print(response.css('[style]'))
print('='*100)
# 選取屬性id值為images-1的元表
print(response.css('[id="images-1"]'))
print('='*100)
# 選取每個div的第一個a
print(response.css('div>a:nth-child(1)'))
print('='*100)


['<img src="imge1.jpg">', '<img src="imge2.jpg">', '<img src="imge3.jpg">']
['<base href="http://example.com/">', '<title>Example website</title>']
[<Selector xpath='descendant-or-self::div/descendant-or-self::*/img' data='<img src="imge1.jpg">'>, <Selector xpath='descendant-or-self::div/descendant-or-self::*/img' data='<img src="imge2.jpg">'>, <Selector xpath='descendant-or-self::div/descendant-or-self::*/img' data='<img src="imge3.jpg">'>]
[<Selector xpath='descendant-or-self::body/div' data='<div id="images-1" style="width: 1230...'>, <Selector xpath='descendant-or-self::body/div' data='<div id="images-2" class="small">\n   ...'>]
[<Selector xpath='descendant-or-self::*[@style]' data='<div id="images-1" style="width: 1230...'>]
[<Selector xpath="descendant-or-self::*[@id = 'images-1']" data='<div id="images-1" style="width: 1230...'>]
[<Selector xpath='descendant-or-self::div/a[count(preceding-sibling::*) = 0]' data='<a href="image1.html">Name: Image1 <b...'>, <Selector xpath='desce

In [205]:
# 選取第二個div的第一個a
print(response.css('div:nth-child(2)>a:nth-child(1)'))
print('='*100)
# 選取第一個div中的最後一個a
print(response.css('div:first-child>a:last-child'))
print('='*100)
# 選取所有a的文字
print(response.css('a::text').extract)

[<Selector xpath='descendant-or-self::div[count(preceding-sibling::*) = 1]/a[count(preceding-sibling::*) = 0]' data='<a href="image4.html">Name: Image 4 <...'>]
[<Selector xpath='descendant-or-self::div[count(preceding-sibling::*) = 0]/a[count(following-sibling::*) = 0]' data='<a href="image1.html">Name: Image3 <b...'>]
<bound method SelectorList.getall of [<Selector xpath='descendant-or-self::a/text()' data='Name: Image1 '>, <Selector xpath='descendant-or-self::a/text()' data='Name: Image2 '>, <Selector xpath='descendant-or-self::a/text()' data='Name: Image3 '>, <Selector xpath='descendant-or-self::a/text()' data='Name: Image 4 '>, <Selector xpath='descendant-or-self::a/text()' data='Name: Image 5 '>]>
